In [0]:
%pip install azure-eventhub


In [0]:
%restart_python

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Azure Event Hub Connection with Key Vault - Databricks
# MAGIC 
# MAGIC **Event Hub Details:**
# MAGIC - Namespace: ehns-gdctraining.servicebus.windows.net
# MAGIC - Event Hub: evh-gdctraining-001
# MAGIC - Permissions: GDCDBXTrainingGroup1 has Azure Event Hubs Data Sender and Data Receiver permissions
# MAGIC 
# MAGIC **Key Vault Secrets:**
# MAGIC - Read Access: evh-gdctraining-001-read
# MAGIC - Write Access: evh-gdctraining-001-write
# MAGIC 
# MAGIC This notebook demonstrates how to connect to Azure Event Hub using Key Vault for secure credential management

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Setup Azure Key Vault Connection
# MAGIC 
# MAGIC **Prerequisites:**
# MAGIC 1. Azure Key Vault must be created
# MAGIC 2. Databricks must have access to Key Vault (via Service Principal or Managed Identity)
# MAGIC 3. Create a secret scope in Databricks linked to your Key Vault

# COMMAND ----------

# MAGIC %md
# MAGIC ### Create Databricks Secret Scope (One-time setup)
# MAGIC 
# MAGIC **Method 1: Using Databricks CLI**
# MAGIC ```bash
# MAGIC databricks secrets create-scope --scope <scope-name> --scope-backend-type AZURE_KEYVAULT \
# MAGIC   --resource-id <key-vault-resource-id> \
# MAGIC   --dns-name <key-vault-dns-name>
# MAGIC ```
# MAGIC 
# MAGIC **Method 2: Using Databricks UI**
# MAGIC 1. Navigate to: `https://<databricks-instance>#secrets/createScope`
# MAGIC 2. Enter Scope Name: `evh-keyvault-scope`
# MAGIC 3. Select: Azure Key Vault
# MAGIC 4. Enter DNS Name: `https://<your-keyvault-name>.vault.azure.net/`
# MAGIC 5. Enter Resource ID: `/subscriptions/<sub-id>/resourceGroups/<rg>/providers/Microsoft.KeyVault/vaults/<keyvault-name>`

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Configuration Setup

# COMMAND ----------

# Event Hub Configuration
eh_namespace = "evhns-gdctraining.servicebus.windows.net"
eh_name = "evh-gdctraining-001"

# Key Vault Secret Scope (replace with your actual scope name)
keyvault_scope = "dbx-ss-kv-gdctraining"  # Replace with your Databricks secret scope name

# Secret names in Key Vault
read_secret_name = "evh-gdctraining-001-read"
write_secret_name = "evh-gdctraining-001-write"

print("Configuration loaded successfully")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Retrieve Access Keys from Key Vault

# COMMAND ----------

# Retrieve the access keys from Key Vault via Databricks Secrets
try:
    read_access_key = dbutils.secrets.get(scope=keyvault_scope, key=read_secret_name)
    write_access_key = dbutils.secrets.get(scope=keyvault_scope, key=write_secret_name)
    print("✓ Successfully retrieved secrets from Key Vault")
except Exception as e:
    print(f"✗ Error retrieving secrets: {str(e)}")
    print("\nTroubleshooting:")
    print("1. Verify secret scope exists: dbutils.secrets.listScopes()")
    print("2. Verify secrets exist in scope: dbutils.secrets.list('<scope-name>')")
    print("3. Check Key Vault access permissions")

# COMMAND ----------

In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Azure Event Hub Connection with Key Vault - Databricks
# MAGIC 
# MAGIC **Event Hub Details:**
# MAGIC - Namespace: ehns-gdctraining.servicebus.windows.net
# MAGIC - Event Hub: evh-gdctraining-001
# MAGIC - Permissions: GDCDBXTrainingGroup1 has Azure Event Hubs Data Sender and Data Receiver permissions
# MAGIC 
# MAGIC **Key Vault Secrets:**
# MAGIC - Read Access: evh-gdctraining-001-read
# MAGIC - Write Access: evh-gdctraining-001-write
# MAGIC 
# MAGIC This notebook demonstrates how to connect to Azure Event Hub using Key Vault for secure credential management

# COMMAND ----------

# MAGIC %md
# MAGIC ## 1. Setup Azure Key Vault Connection
# MAGIC 
# MAGIC **Prerequisites:**
# MAGIC 1. Azure Key Vault must be created
# MAGIC 2. Databricks must have access to Key Vault (via Service Principal or Managed Identity)
# MAGIC 3. Create a secret scope in Databricks linked to your Key Vault

# COMMAND ----------

# MAGIC %md
# MAGIC ### Create Databricks Secret Scope (One-time setup)
# MAGIC 
# MAGIC **Method 1: Using Databricks CLI**
# MAGIC ```bash
# MAGIC databricks secrets create-scope --scope <scope-name> --scope-backend-type AZURE_KEYVAULT \
# MAGIC   --resource-id <key-vault-resource-id> \
# MAGIC   --dns-name <key-vault-dns-name>
# MAGIC ```
# MAGIC 
# MAGIC **Method 2: Using Databricks UI**
# MAGIC 1. Navigate to: `https://<databricks-instance>#secrets/createScope`
# MAGIC 2. Enter Scope Name: `evh-keyvault-scope`
# MAGIC 3. Select: Azure Key Vault
# MAGIC 4. Enter DNS Name: `https://<your-keyvault-name>.vault.azure.net/`
# MAGIC 5. Enter Resource ID: `/subscriptions/<sub-id>/resourceGroups/<rg>/providers/Microsoft.KeyVault/vaults/<keyvault-name>`

# COMMAND ----------

# MAGIC %md
# MAGIC ## 2. Configuration Setup

# COMMAND ----------

# Event Hub Configuration
eh_namespace = "evhns-gdctraining.servicebus.windows.net"
eh_name = "evh-gdctraining-001"

# Key Vault Secret Scope (replace with your actual scope name)
keyvault_scope = "dbx-ss-kv-gdctraining"  # Replace with your Databricks secret scope name

# Secret names in Key Vault
read_secret_name = "evh-gdctraining-001-read"
write_secret_name = "evh-gdctraining-001-write"

print("Configuration loaded successfully")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 3. Retrieve Access Keys from Key Vault

# COMMAND ----------

# Retrieve the access keys from Key Vault via Databricks Secrets
try:
    read_access_key = dbutils.secrets.get(scope=keyvault_scope, key=read_secret_name)
    write_access_key = dbutils.secrets.get(scope=keyvault_scope, key=write_secret_name)
    print("✓ Successfully retrieved secrets from Key Vault")
except Exception as e:
    print(f"✗ Error retrieving secrets: {str(e)}")
    print("\nTroubleshooting:")
    print("1. Verify secret scope exists: dbutils.secrets.listScopes()")
    print("2. Verify secrets exist in scope: dbutils.secrets.list('<scope-name>')")
    print("3. Check Key Vault access permissions")

# COMMAND ----------

# MAGIC %md
# MAGIC ## 4. Build Connection Strings

# COMMAND ----------

# Connection String for READING from Event Hub
#read_connection_string = f"Endpoint=sb://{eh_namespace}/;SharedAccessKeyName=evhaccesspolicylisten;SharedAccessKey={read_access_key};EntityPath={eh_name}"
read_connection_string = f"Endpoint=sb://evhns-gdctraining.servicebus.windows.net/;SharedAccessKeyName=evhaccesspolicylisten;SharedAccessKey=<fromkeyvault>;EntityPath=evh-gdctraining-001"

# Connection String for WRITING to Event Hub
#write_connection_string = f"Endpoint=sb://{eh_namespace}/;SharedAccessKeyName=evhaccesspolicysend;SharedAccessKey={write_access_key};EntityPath={eh_name}"
write_connection_string = f"Endpoint=sb://evhns-gdctraining.servicebus.windows.net/;SharedAccessKeyName=evhaccesspolicysend;SharedAccessKey=<fromkeyvault>;EntityPath=evh-gdctraining-001"

